In [1]:
import keras
import tensorflow as tf

from keras.applications import ResNet50, VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Flatten, Dense, Input, Dropout
from keras.layers import Conv2D, ZeroPadding2D
from keras.layers import MaxPooling2D
from keras.metrics import top_k_categorical_accuracy
from keras.models import Model
from keras.optimizers import SGD
from keras import backend as K

Using TensorFlow backend.


In [2]:
train_datagen = ImageDataGenerator(
        samplewise_center=True,
        samplewise_std_normalization=True,
        horizontal_flip=True,
        vertical_flip=True,
        width_shift_range=0.1,
        height_shift_range=0.1)

test_datagen = ImageDataGenerator()

In [3]:
train_generator = train_datagen.flow_from_directory(
    '../data/data_train/',     
    class_mode='categorical',           
    target_size=(224, 224),
    batch_size=50)

validation_generator = test_datagen.flow_from_directory(
    '../data/data_test/',  
    class_mode='categorical',
    target_size=(224, 224),
    batch_size=50)

Found 24580 images belonging to 257 classes.
Found 6027 images belonging to 257 classes.


In [4]:
tbCallBack = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=True)

In [5]:
#Get back the convolutional part of a VGG network trained on ImageNet
model_vgg16_conv = VGG16(weights='imagenet', include_top=False)
model_vgg16_conv.layers[1].trainable = False
model_vgg16_conv.layers[2].trainable = False
model_vgg16_conv.layers[4].trainable = False
model_vgg16_conv.layers[5].trainable = False
model_vgg16_conv.layers[7].trainable = False
model_vgg16_conv.layers[8].trainable = False
model_vgg16_conv.layers[9].trainable = False
model_vgg16_conv.layers[11].trainable = False
model_vgg16_conv.layers[12].trainable = False
model_vgg16_conv.layers[13].trainable = False
model_vgg16_conv.layers[15].trainable = False
model_vgg16_conv.layers[16].trainable = False
model_vgg16_conv.layers[18].trainable = False

input = Input(shape=(224, 224, 3), name='image_input')

output_vgg16_conv = model_vgg16_conv(input)

x = Flatten(name='flatten')(output_vgg16_conv)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(4096, activation='relu', name='fc2')(x)
x = Dense(257, activation='softmax', name='predictions')(x)

model = Model(inputs = input, outputs=x)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 224, 224, 3)       0         
_________________________________________________________________
vgg16 (Model)                multiple                  14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
fc1 (Dense)                  (None, 4096)              102764544 
_________________________________________________________________
fc2 (Dense)                  (None, 4096)              16781312  
_________________________________________________________________
predictions (Dense)          (None, 257)               1052929   
Total params: 135,313,473
Trainable params: 122,958,593
Non-trainable params: 12,354,880
_____________________________________________________

In [6]:
def top_5_categorical_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=5)

In [7]:
sgd = SGD(lr=0.01, decay=0.0005, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy', top_5_categorical_accuracy])

In [8]:
model.fit_generator(
    generator=train_generator,
    steps_per_epoch=492,
    validation_data=validation_generator,
    validation_steps=121,
    epochs=10,
    callbacks=[tbCallBack],
    verbose=2)

Epoch 1/10
619s - loss: 4.5228 - acc: 0.1377 - top_5_categorical_accuracy: 0.2605 - val_loss: 13.6662 - val_acc: 0.1233 - val_top_5_categorical_accuracy: 0.3133
Epoch 2/10
589s - loss: 3.6294 - acc: 0.2412 - top_5_categorical_accuracy: 0.4409 - val_loss: 13.0863 - val_acc: 0.1626 - val_top_5_categorical_accuracy: 0.4740
Epoch 3/10
591s - loss: 3.1927 - acc: 0.3102 - top_5_categorical_accuracy: 0.5304 - val_loss: 12.8533 - val_acc: 0.1862 - val_top_5_categorical_accuracy: 0.5565
Epoch 4/10
593s - loss: 2.8852 - acc: 0.3619 - top_5_categorical_accuracy: 0.5926 - val_loss: 12.6126 - val_acc: 0.1989 - val_top_5_categorical_accuracy: 0.6195
Epoch 5/10
611s - loss: 2.5974 - acc: 0.4127 - top_5_categorical_accuracy: 0.6454 - val_loss: 12.5824 - val_acc: 0.2031 - val_top_5_categorical_accuracy: 0.6894
Epoch 6/10
611s - loss: 2.3466 - acc: 0.4572 - top_5_categorical_accuracy: 0.6935 - val_loss: 12.3122 - val_acc: 0.2235 - val_top_5_categorical_accuracy: 0.7015
Epoch 7/10
611s - loss: 2.1189 - a

In [9]:
model.save('../data/vgg16_model_trained_augmentation_light.h5')